In [24]:
import numpy as np
import pandas as pd

from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_absolute_error

In [84]:
# Создаём сэмпл: лаботаторные отделения, в которых ведут прием врачи по специализациям sp1, sp2, sp3
n_samples = 500

# расстояние до метро
dist_to_metro = np.random.choice(5, n_samples)
# количество лицензий лабораторного отделения
emp = np.random.randint(0, 3, n_samples) + 1
# кабинеты врачей 1,2,3
sp1 = np.random.randint(1, 2, n_samples)
sp2 = np.random.randint(0, 2, n_samples)
sp3 = np.random.randint(0, 2, n_samples)
# сколько дней уже открыто отделение
working_days = np.random.choice(3660, n_samples)
# количество посетителей в день
visitors = 5 *((sp1 + sp2 + sp3)*(sp1 + sp2 + sp3)) - 0.01 * dist_to_metro + 0.05 * working_days
av_bill = 1300 + np.random.randint(-500, 500, n_samples)
earnings = visitors * av_bill 

data = pd.DataFrame({'dist_to_metro': dist_to_metro, 'working_days': working_days, 'av_bill': av_bill, 'earnings': earnings, 'emp': emp, 'sp1': sp1, 'sp2': sp2,'sp3': sp3, 'visitors': visitors})
data.head(5)



,dist_to_metro,working_days,av_bill,earnings,emp,sp1,sp2,sp3,visitors
0,1,2171,1684,216461.36,1,1,1,0,128.54
1,4,1413,1257,145321.77,3,1,1,1,115.61
2,2,680,1626,87771.48,3,1,1,0,53.98
3,2,2717,1724,242790.92,2,1,0,0,140.83
4,4,2608,1281,192611.16,1,1,1,0,150.36


In [ ]:
# строю модель по прогнозированию выручки earnings на всех переменных

In [86]:
X = data[['dist_to_metro', 'working_days', 'visitors','emp', 'sp1', 'sp2', 'sp3']]
y = data['earnings']
reg = LinearRegression().fit(X, y)
print('Weights: {}'.format(reg.coef_))
print('Bias: {}'.format(reg.intercept_))

pred_values = reg.predict(data[['dist_to_metro', 'working_days', 'visitors', 'emp', 'sp1', 'sp2', 'sp3']])
print('Error: {}'.format(mean_absolute_error(pred_values, y)))

Weights: [-3490.80351926   -21.246487    1718.89417136 -1598.08080428
     0.         -9661.51947207 -4080.20311545]
Bias: 6496.3724932062905
Error: 27819.21969946049


In [ ]:
# убираю количество лицензий emp
# и вижу, что ошибка стала не хуже

In [77]:
X = data[['dist_to_metro', 'working_days', 'visitors', 'sp1', 'sp2', 'sp3']]
y = data['earnings']
reg = LinearRegression().fit(X, y)
print('Weights: {}'.format(reg.coef_))
print('Bias: {}'.format(reg.intercept_))

pred_values = reg.predict(data[['dist_to_metro', 'working_days', 'visitors', 'sp1', 'sp2', 'sp3']])
print('Error: {}'.format(mean_absolute_error(pred_values, y)))

Weights: [ 1.45572119e+02  2.17813402e+01  8.36420439e+02 -3.91082722e-11
  6.03418360e+03  1.41033203e+04]
Bias: 1311.842120246467
Error: 28102.667162745438


In [78]:
data['all_sp'] = data['sp1'] + data['sp2'] + data['sp3']
data.head()

,dist_to_metro,working_days,av_bill,earnings,emp,sp1,sp2,sp3,visitors,all_sp
0,1,1410,1653,149579.97,3,1,0,1,90.49,2
1,0,2774,982,180393.40,3,1,1,1,183.70,3
2,3,1419,1797,163383.24,1,1,0,1,90.92,2
3,1,2120,1187,131745.13,2,1,0,0,110.99,1
4,3,3107,1366,273637.12,1,1,1,1,200.32,3


In [79]:
X = data[['dist_to_metro', 'working_days', 'visitors', 'all_sp']]
y = data['earnings']
reg = LinearRegression().fit(X, y)
print('Weights: {}'.format(reg.coef_))
print('Bias: {}'.format(reg.intercept_))

pred_values = reg.predict(data[['dist_to_metro', 'working_days', 'visitors', 'all_sp']])
print('Error: {}'.format(mean_absolute_error(pred_values, y)))

Weights: [  132.43432006    22.02020464   833.43517617 10136.98769307]
Bias: -8950.778118284914
Error: 28082.96172369431


In [81]:
X = data[['visitors', 'all_sp']]
y = data['earnings']
reg = LinearRegression().fit(X, y)
print('Weights: {}'.format(reg.coef_))
print('Bias: {}'.format(reg.intercept_))

pred_values = reg.predict(data[['visitors', 'all_sp']])
print('Error: {}'.format(mean_absolute_error(pred_values, y)))

Weights: [1272.88065175 1267.00125929]
Bias: -782.381433265371
Error: 28121.51298530019


In [15]:
from sklearn.metrics import mean_absolute_error

X = data[['dist_to_metro', 'sp1', 'sp2', 'sp3']]
y = data['visitors']
reg = LinearRegression().fit(X, y)
print('Weights: {}'.format(reg.coef_))
print('Bias: {}'.format(reg.intercept_))

pred_values = reg.predict(data[['dist_to_metro', 'sp1', 'sp2', 'sp3']])
print('Error: {}'.format(mean_absolute_error(pred_values, y)))

Weights: [3.77436839e+00 2.13162821e-14 1.73420194e+01 2.01275966e+01]
Bias: 88.91042779063474
Error: 45.72665684225884


In [128]:
# Создаем новый признак "Количество специалистов в отделении"
data['num_sp'] = data['sp1'] + data['sp2'] + data['sp3']
data.head(5)

,dist_to_metro,emp,sp1,sp2,sp3,visitors,num_sp
0,0,1,1,0,1,120000,2
1,1,3,1,0,0,29000,1
2,3,3,1,1,0,117000,2
3,2,2,1,1,1,268000,3
4,0,1,1,1,1,270000,3


In [129]:
X = data[['dist_to_metro', 'num_sp']]
y = data['visitors']

reg = LinearRegression().fit(X, y)
print('Weights: {}'.format(reg.coef_))
print('Bias: {}'.format(reg.intercept_))

pred_values = reg.predict(data[['dist_to_metro', 'num_sp']])
print('Error: {}'.format(mean_absolute_error(pred_values, y)))

Weights: [ -1145.49746079 119767.98367946]
Bias: -103912.14512607997
Error: 14986.955069828115
